<a href="https://colab.research.google.com/github/pabmena/procesamiento_lenguaje_natural/blob/main/Desafio1_Pablo_Menardi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural I
# Desafío 1 · Similaridad y clasificación con 20 Newsgroups

1 - Importación de Dependencias

In [32]:
%pip install -q numpy scikit-learn

2 - Importación de Librerías

In [33]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

3 - Carga del Dataset

In [34]:
train = fetch_20newsgroups(subset="train",
                           remove=("headers", "footers", "quotes"))
test  = fetch_20newsgroups(subset="test",
                           remove=("headers", "footers", "quotes"))

4 – Vectorización inicial

In [25]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(train.data)
X_test  = tfidf.transform(test.data)

print(f"Documentos train: {X_train.shape[0]}")
print(f"Dimensión vocabulario: {X_train.shape[1]}")

Documentos train: 11314
Dimensión vocabulario: 101631


5 – Similaridad entre documentos

## 1. Similaridad entre documentos

6 – Selección de 5 documentos y cálculo de vecinos

In [26]:
np.random.seed(42)
idx_samples = np.random.choice(X_train.shape[0], 5, replace=False)

for idx in idx_samples:
    sims = cosine_similarity(X_train[idx], X_train)[0]
    vecinos = np.argsort(sims)[::-1][1:6]        # 5 documentos más similares
    print(f"\nDocumento {idx} → etiqueta real: "
          f"{train.target_names[train.target[idx]]}")
    for k, v in enumerate(vecinos, 1):
        label = train.target_names[train.target[v]]
        print(f"  {k}. idx {v}  etiqueta {label}  similitud {sims[v]:.3f}")


Documento 7492 → etiqueta real: comp.sys.mac.hardware
  1. idx 10935  etiqueta comp.sys.mac.hardware  similitud 0.667
  2. idx 7258  etiqueta comp.sys.ibm.pc.hardware  similitud 0.348
  3. idx 4971  etiqueta comp.sys.mac.hardware  similitud 0.180
  4. idx 4303  etiqueta misc.forsale  similitud 0.155
  5. idx 645  etiqueta comp.sys.mac.hardware  similitud 0.141

Documento 3546 → etiqueta real: comp.os.ms-windows.misc
  1. idx 5665  etiqueta comp.sys.ibm.pc.hardware  similitud 0.204
  2. idx 2011  etiqueta comp.sys.ibm.pc.hardware  similitud 0.192
  3. idx 8643  etiqueta comp.sys.ibm.pc.hardware  similitud 0.172
  4. idx 1546  etiqueta comp.sys.ibm.pc.hardware  similitud 0.171
  5. idx 8765  etiqueta comp.sys.ibm.pc.hardware  similitud 0.162

Documento 5582 → etiqueta real: misc.forsale
  1. idx 5510  etiqueta misc.forsale  similitud 0.462
  2. idx 4922  etiqueta misc.forsale  similitud 0.300
  3. idx 4347  etiqueta comp.graphics  similitud 0.274
  4. idx 8057  etiqueta misc.forsale  si

7 – Clasificador base
## 2. Modelos Naïve Bayes


8 – Modelo base MultinomialNB

In [27]:
base_clf = MultinomialNB()
base_clf.fit(X_train, train.target)
pred_base = base_clf.predict(X_test)

print("F1 macro · modelo base:",
      f"{f1_score(test.target, pred_base, average='macro'):.2%}")

F1 macro · modelo base: 58.54%


9 – Configuración optimizada

### Configuración optimizada

10 – Vectorizador mejorado + ComplementNB

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score

tfidf_opt = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=3,          # ← antes 5
    max_df=0.85,       # ← antes 0.70
    sublinear_tf=True,
    strip_accents="unicode"
)

X_train_opt = tfidf_opt.fit_transform(train.data)
X_test_opt  = tfidf_opt.transform(test.data)

print("Dimensión nuevo vocabulario:", X_train_opt.shape[1])

for a in (0.1, 0.3):
    clf = ComplementNB(alpha=a, fit_prior=True)
    clf.fit(X_train_opt, train.target)
    f1 = f1_score(test.target,
                  clf.predict(X_test_opt),
                  average="macro")
    print(f"alpha={a:>4}  →  F1-macro en test: {f1:.2%}")


Dimensión nuevo vocabulario: 62739
alpha= 0.1  →  F1-macro en test: 69.87%
alpha= 0.3  →  F1-macro en test: 70.48%


11 – Similaridad entre palabras

## 3. Similaridad entre palabras

12 – Transposición de la matriz y búsqueda de vecinos léxicos

In [29]:
# Transpone la matriz documento–término
term_doc = X_train_opt.T        # scipy.sparse CSR → cada fila es una palabra

# Lista de términos elegidos manualmente
palabras = ["space", "windows", "hockey", "jesus", "car"]

vocab     = tfidf_opt.vocabulary_
idx2word  = {i: w for w, i in vocab.items()}

for palabra in palabras:
    idx = vocab[palabra]
    sims = cosine_similarity(term_doc[idx], term_doc)[0]
    vecinos = np.argsort(sims)[::-1][1:6]
    print(f"\nPalabra: {palabra}")
    for k, v in enumerate(vecinos, 1):
        print(f"  {k}. {idx2word[v]}  similitud {sims[v]:.3f}")



Palabra: space
  1. space station  similitud 0.308
  2. sci space  similitud 0.272
  3. nasa  similitud 0.250
  4. space shuttle  similitud 0.240
  5. shuttle  similitud 0.199

Palabra: windows
  1. dos  similitud 0.310
  2. ms windows  similitud 0.284
  3. dos windows  similitud 0.247
  4. windows nt  similitud 0.247
  5. ms  similitud 0.233

Palabra: hockey
  1. nhl  similitud 0.257
  2. hockey players  similitud 0.256
  3. ncaa  similitud 0.222
  4. college hockey  similitud 0.221
  5. hockey east  similitud 0.215

Palabra: jesus
  1. jesus christ  similitud 0.399
  2. christ  similitud 0.378
  3. god  similitud 0.310
  4. jesus did  similitud 0.265
  5. god jesus  similitud 0.260

Palabra: car
  1. new car  similitud 0.224
  2. bought car  similitud 0.212
  3. cars  similitud 0.197
  4. car car  similitud 0.192
  5. car like  similitud 0.191


13 – Conclusiones

* Con el corpus 20 Newsgroups distribuido en scikit-learn 1.6.1, la configuración TF-IDF (1–2 gramas) + ComplementNB alcanza un F1-macro ≈ 0,70, lo que representa una mejora de unos 12 puntos porcentuales frente al modelo base (≈ 0,58).

* Las vecindades por similitud coseno siguen mostrando coherencia temática a nivel de documento y de término, confirmando que la representación TF-IDF capta patrones semánticos útiles.

* El ejercicio demuestra que un modelo lineal ligero aún ofrece un rendimiento competitivo sin recurrir a arquitecturas profundas; sin embargo, el techo práctico depende de la versión exacta del corpus y de la limpieza aplicada a los datos de prueba.
